In [1]:
# Imports
import pandas as pd
import numpy as np
import random
import json
import ast

# Tests

In [2]:
headers = ["name","health","armor_class","initiative","initiative_bonus","team","group"]
turn_track = pd.DataFrame(columns=headers)
turn_track

,name,health,armor_class,initiative,initiative_bonus,team,group


In [3]:
character = {
    "name":"Mr.",
    "health":56,
    "armor_class":20,
    "initiative": 15,
    "initiative_bonus":2,
    "team":"Heroes",
    "group":"Alone"
}
turn_track.append(character,ignore_index=True)


,name,health,armor_class,initiative,initiative_bonus,team,group
0,Mr.,56,20,15,2,Heroes,Alone


In [4]:
headers = ["name","health","armor_class"]
filter_track = pd.DataFrame(columns=headers)
character = {
    "name":"Mrs.",
    "health":568,
    "armor_class":200
}
filter_track = filter_track.append(character,ignore_index=True)
# filter_track
filter_track[filter_track.columns[[True,False,True]]]

,name,armor_class
0,Mrs.,200


In [5]:
party1 = pd.read_csv(".\Party1.csv")
party1["group"] = np.nan

In [6]:
def read_import(path,import_groups=True):
    party = pd.read_csv(path)
    if import_groups and ('group' in list(party)):
        return party
    else:
        party["group"] = np.nan
        return party
party2=read_import(".\Party2.csv")
party2.iloc[0]['group']

'Winterfell'

In [7]:
def initiative_based_group_assignment(groups:pd.DataFrame):
    df = groups.copy()
    df['total_initiative'] = df['initiative']+df['initiative_bonus']
    df.sort_values(by='total_initiative',ascending=False,inplace=True)
    df_count = {}
    team_order = df['team']
    group_placement = []
    lastTeam = None
    for team in team_order:
        if team != lastTeam :
            if team in df_count.keys() :
                df_count[team]+=1
            else :
                df_count[team]=1
            lastTeam = team
        group_placement.append(f"{team} {df_count[team]}")
    df['group']=group_placement
    return df.drop(columns="total_initiative")
party2_grouped = initiative_based_group_assignment(party2)
# party2_grouped

In [8]:
def roll(sided=20):
    return random.randint(1,sided)

def auto_initiative(groups:pd.DataFrame):
    df = groups.copy()
    df['initiative'] = df['initiative'].apply(lambda x: roll(20))
    return df
party2_random_init = auto_initiative(party2)
party2_random_init_grouped = initiative_based_group_assignment(party2_random_init)
# party2_random_init_grouped

In [9]:
def groups_list(turn_track:pd.DataFrame):
    return(turn_track['group'].unique())
# groups_list(party2_random_init_grouped)

In [10]:
def df_match_slice(df:pd.DataFrame,column,match):
    return df[df[column]==match]
group_to_split_df = df_match_slice(party2_random_init_grouped,'group','Stark 3')
# for member in group_to_split_df.index:
#     print(type(member))

In [11]:
party2_change_init = party2_random_init_grouped.copy()
character_selected = 'Eddard "Ned" Stark'
party2_change_init.loc[(party2_change_init['name'] == character_selected,'initiative')]=24
# party2_change_init

In [12]:
def df_set_match_slice(df:pd.DataFrame,column,match,new_data):
    df_slice = df[df[column] == match].copy()
    df_slice[column] = new_data
    df_copy = df.copy()
    df_copy.update(df_slice)
    return df_copy
# group_to_split_df = df_set_match_slice(party2_random_init_grouped,'group','Targaryen 1',['Test1','Test2'])
# group_to_split_df

In [13]:
type(group_to_split_df)

pandas.core.frame.DataFrame

In [14]:
def move_group(groups:pd.DataFrame,group_to_move,before_or_after,group_to_place):
    df = groups.copy()
    df.reset_index(drop=True,inplace=True)
    slice_group_to_move = df[df['group']==group_to_move].copy()
    df.drop(slice_group_to_move.index,inplace=True)
    df.reset_index(drop=True,inplace=True)
    slice_group_to_move.reset_index(drop=True,inplace=True)
    if before_or_after == "Before" :
        index_split_point = (df[df['group']==group_to_place].index[0]) #first index
    else :
        index_split_point = df[df['group']==group_to_place].index[-1]+1 #last index
    return pd.concat([df.iloc[:index_split_point],slice_group_to_move,df.iloc[index_split_point:]]).reset_index(drop=True)
move_test = party2_random_init_grouped.copy()
# move_test
move_group(move_test,"Stark 3","After","Targaryen 2")

,name,health,armor_class,initiative,initiative_bonus,team,group,attributes
0,Viserys Targaryen,90,16,20,3,Targaryen,Targaryen 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
1,Robert Baratheon,90,16,19,3,Stark,Stark 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
2,Cersei Lannister,90,16,19,3,Lannister,Lannister 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
3,Jaime Lannister,90,16,17,3,Lannister,Lannister 1,"{'weapon': ['dagger','sword','metal hand']}"
4,Robb Stark,90,16,13,3,Stark,Stark 2,NaN
5,"Eddard ""Ned"" Stark",90,13,12,3,Stark,Stark 2,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
6,Arya Stark,90,16,10,3,Stark,Stark 2,"{'weapon': ['dagger','needle']}"
7,Catelyn Stark,90,16,8,3,Stark,Stark 2,"{'weapon': ['dagger'], 'feats': 'deadman'}"
8,Daenerys Targaryen,90,16,6,3,Targaryen,Targaryen 2,"{'weapon': ['dagger'], 'feats': 'deadman','dra..."
9,Sansa Stark,90,16,5,3,Stark,Stark 3,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."


In [15]:
def move_character(groups:pd.DataFrame,person_to_move,destination_group):
    df = groups.copy()
    df.reset_index(drop=True,inplace=True)
    slice_character_to_move = df[df['name']==person_to_move].copy()
    slice_character_to_move['group']=destination_group
    df.drop(slice_character_to_move.index,inplace=True)
    index_split_point = df[df['group']==destination_group].index[-1]+1 #last index
    return pd.concat([df.iloc[:index_split_point],slice_character_to_move,df.iloc[index_split_point:]]).reset_index(drop=True)
move_char_test = party2_random_init_grouped.copy()
# move_test
# move_character(move_test,"Arya Stark","Stark 2")

In [16]:
def move_character_to_new_group(groups:pd.DataFrame,person_to_move,new_group_name):
    df = groups.copy()
    old_group=df.loc[(df["name"]==person_to_move,"group")].values[0]
    df.loc[(df["name"]==person_to_move,"group")]=new_group_name
    if df[df['group']==old_group].empty :
        return df # If character was the only member of a group, no need to rearrange
    return move_group(df,new_group_name,"After",old_group)
move_char_new_group_test = party2_random_init_grouped.copy()

# move_character_to_new_group(move_char_new_group_test,"Cersei Lannister","Kinda Annoying")


In [17]:
def merge_groups(groups:pd.DataFrame,merge_group_1,merge_group_2,merged_name):
    df = groups.copy()
    df = move_group(df,merge_group_1,"After",merge_group_2)
    df['group'].replace([merge_group_1,merge_group_2],[merged_name,merged_name],inplace=True)
    return df

def individual_groups(groups:pd.DataFrame):
    df = groups.copy()
    df['group']=df['name']
    return df

merge_test = party2_random_init_grouped.copy()
merge_test = individual_groups(merge_test)
merge_test
merge_groups(merge_test,'Eddard "Ned" Star',"Catelyn Stark","Stark Parents")

,name,health,armor_class,initiative,initiative_bonus,team,group,attributes
0,Viserys Targaryen,90,16,20,3,Targaryen,Viserys Targaryen,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
1,Robert Baratheon,90,16,19,3,Stark,Robert Baratheon,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
2,Cersei Lannister,90,16,19,3,Lannister,Cersei Lannister,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
3,Jaime Lannister,90,16,17,3,Lannister,Jaime Lannister,"{'weapon': ['dagger','sword','metal hand']}"
4,Robb Stark,90,16,13,3,Stark,Robb Stark,NaN
5,"Eddard ""Ned"" Stark",90,13,12,3,Stark,"Eddard ""Ned"" Stark","{'weapon': ['dagger','sword'], 'feats': 'deadm..."
6,Arya Stark,90,16,10,3,Stark,Arya Stark,"{'weapon': ['dagger','needle']}"
7,Catelyn Stark,90,16,8,3,Stark,Stark Parents,"{'weapon': ['dagger'], 'feats': 'deadman'}"
8,Daenerys Targaryen,90,16,6,3,Targaryen,Daenerys Targaryen,"{'weapon': ['dagger'], 'feats': 'deadman','dra..."
9,Sansa Stark,90,16,5,3,Stark,Sansa Stark,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."


In [18]:
groups_list(party2)[groups_list(party2)!='Winterfell']

array(['Kings Landing', 'Nights Watch'], dtype=object)

In [19]:
def read_audit(path):
    audit_read = pd.read_csv(path)

    audit_tags = audit_read[audit_read['Audit Header'].str.contains('tags_')].reset_index(drop=True)
    audit_out = audit_read[audit_read['Audit Header'].str.contains('out_')].reset_index(drop=True)
    audit_meta = audit_read[audit_read['Audit Header'].str.contains('meta_')].reset_index(drop=True)
    
    audit_actions = audit_read.drop(index=audit_read[audit_read['Audit Header'].str.contains('tags_')].index)
    audit_actions = audit_actions.drop(index=audit_read[audit_read['Audit Header'].str.contains('out_')].index)
    audit_actions = audit_actions.drop(index=audit_read[audit_read['Audit Header'].str.contains('meta_')].index)

    audit_actions.reset_index(drop=True,inplace=True)

    audit_tags['Audit Header'] = audit_tags['Audit Header'].str[5:]
    audit_out['Audit Header'] = audit_out['Audit Header'].str[4:]
    audit_meta['Audit Header'] = audit_meta['Audit Header'].str[5:]

    return process_audit(audit_actions), process_audit(audit_out), process_audit(audit_tags), process_audit(audit_meta)

def process_audit(df:pd.DataFrame):
    df = df.set_index("Audit Header").transpose().reset_index(drop=True)
    audit_out = {}
    for col in df.columns : audit_out[col] = df[col].dropna().to_list()
    return audit_out

audit_actions,audit_outcome,audit_tags,audit_meta = read_audit("..\data\default_audit_actions.csv")

In [20]:
def next_turn(groups:pd.DataFrame,current_turn):
    groups.reset_index(drop=True,inplace=True)
    current_turns_last_index=groups[groups['group']==current_turn].index[-1]
    if current_turns_last_index == len(groups)-1 : #loop around detection
        return groups.iloc[0]['group']
    else:
        return groups.iloc[current_turns_last_index+1]['group']
# party2_random_init_grouped
# next_turn(party2_random_init_grouped,'Stark 4')

In [21]:
def groups_gathered_check(groups:pd.DataFrame):
    order_list = groups_list(groups)
    team_changes = groups["group"].shift() != groups["group"]
    return len(team_changes[team_changes==True])==len(order_list)
# groups_gathered_check(party2)

In [22]:
# party2_random_init_grouped[party2_random_init_grouped['group']=="Stark 2"].iloc[2]['attributes']

In [23]:
# di = {'test':[1,2,3,40],'diction':[5,6,7,8,"nine"],'number':4,'string':'hello'}
# party2_2=party2.copy()
# party2_2['attributes']=[di.copy() for _ in range(len(party2_2))]
# party2_2.to_csv(".\Party2_2.csv")

In [24]:
def attributes_list(groups:pd.DataFrame):
    df = groups[['name','attributes']].copy()
    df.dropna(inplace=True)
    all_attributes = []
    for index,character in df.iterrows():
        character_attributes = ast.literal_eval(character['attributes'])
        for key, values in character_attributes.items() :
            if type(values) == list :
                for item in values:
                    all_attributes.append(f"{character['name']} - {key} - {item}")
            else :
                all_attributes.append(f"{character['name']} - {key} - {values}")
    return all_attributes
# attributes_list(party2_random_init_grouped)

In [50]:
meta = audit_meta.copy()

def parse_meta_str(meta):
    meta_list = meta.strip('][').split(',')
    output = {}
    output["target"] = meta_list[1]
    output["modification"] = meta_list[2]
    output["wording"] = meta_list[0]
    return meta_list[0],output

def meta_to_dict(audit_meta,key='Results'):
    output = {}
    for meta in audit_meta[key]:
        parse_name, parse_dict = parse_meta_str(meta)
        output[parse_name] = parse_dict
    return output

def has_meta(result,meta_lookup:dict):
    return result in meta_lookup.keys()

# has_meta("Dealt Damage",meta_to_dict(meta))


# Possible Classes

In [26]:
class Character:
    name : str
    health : int
    initiative : int
    team : str
    
    def __init__(self):
        self.name = ""
        self.health = 0
        self.initiative = 0
        self.team = None
    def __init__(self,name,health,initiative,team):
        self.name = name
        self.health = health
        self.initiative = initiative
        self.team = team

In [27]:
class GroupLoop:
    group : pd.DataFrame
    name : str

    def __init__(self):
        self.group = pd.DataFrame()
        self.name = "Unnamed Group"
    def __init__(self, list_of_characters,group_name):
        self.group = pd.DataFrame(list_of_characters)
        self.name = group_name
    def set_next_group(self, next_group):
        self.next_group = next_group